In [1]:
import numpy as np
from isaacgym import gymapi, gymtorch
from isaacgym.torch_utils import *
from tqdm import tqdm

Importing module 'gym_38' (/data2/users/nk523/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /data2/users/nk523/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.1.1
Device count 4
/data2/users/nk523/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /data2/users/nk523/.cache/torch_extensions/py38_cu121 as PyTorch extensions root...
Emitting ninja build file /data2/users/nk523/.cache/torch_extensions/py38_cu121/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module gymtorch...


ninja: no work to do.


In [2]:
from franka_panda_env import FrankaPandaEnv
from misc import wrap_angle
from motion_primitive_controller import MotionPrimitiveController
from position_controller import PositionController

In [3]:
env = FrankaPandaEnv(use_gpu=True, use_gpu_pipeline=False, headless=True)

motion_primitive_controller = MotionPrimitiveController(env.states["eef_pos"])
motion_primitive_controller.update(
    current_positions=env.states["eef_pos"],
    current_velocities=env.states["eef_vel"],
    cube_pos_relative=env.states["cube_pos_relative"],
    states_dict=env.states,
    force=True,
)

position_controller = PositionController(env)

Not connected to PVD
+++ Using GPU PhysX
Physics Engine: PhysX
Physics Device: cuda:0
GPU Pipeline: disabled
Loading asset 'urdf/franka_description/robots/franka_panda_gripper.urdf' from 'assets'
Creating 36 environments


In [4]:
frame_num = 0
for i in tqdm(range(100)):
    # Reset all environments
    env.reset_pose("default")
    env.reset_cube()
    motion_primitive_controller.dones *= False
    motion_primitive_controller.update(
        current_positions=env.states["eef_pos"],
        current_velocities=env.states["eef_vel"],
        cube_pos_relative=env.states["cube_pos_relative"],
        states_dict=env.states,
        force=True,
        epsilon=0.5,
    )
    for _ in range(500):
        indices_done = motion_primitive_controller.update(
            current_positions=env.states["eef_pos"],
            current_velocities=env.states["eef_vel"],
            cube_pos_relative=env.states["cube_pos_relative"],
            states_dict=env.states,
            epsilon=0.5,
        )
        position_controller.set_target(
            motion_primitive_controller.current_targets[:, :2]
        )
        position_controller.step()
        # env.gym.write_viewer_image_to_file(env.viewer, f"./images/{frame_num}.png")
        env.reset_pose("default", indices_done)
        env.reset_cube(indices_done)
        motion_primitive_controller.dones *= False
        frame_num += 1
    motion_primitive_controller.save_trajectories(f"{i}.pth")
    motion_primitive_controller.clear_trajectories()

  0%|                                                                                 | 0/100 [00:05<?, ?it/s]


NameError: name 'save_trajectories' is not defined